<a href="https://colab.research.google.com/github/qingyuan-wu/Praxis-III/blob/ML-model/classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plastics Classifier using PlastNet
This program will take in a single image of a plastic and classify it as either PET or non-PET.

### 0. Import Libraries and Mount Drive

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 1. Pre-process Input Image
The trained CNN works on 1277x1277 images. Pre-process input image to have dimensions 1277x1277 pixels.

In [ ]:
!pip install pyheif

     |████████████████████████████████| 9.8 MB 4.4 MB/s 


In [ ]:
from PIL import Image

In [ ]:
def preprocess(img_path, out_path, out_height, out_width):
    import pyheif

    '''
    resize images to out_height x out_width and convert from heic to jpg if necessary
    Save .jpg image to out_path
    '''
    if ".HEIC" in img_path:
        # convert from heic to jpeg
        heif_file = pyheif.read(img_path)
        image = Image.frombytes(
            heif_file.mode, 
            heif_file.size, 
            heif_file.data,
            "raw",
            heif_file.mode,
            heif_file.stride,
            )
        image.save(out_path, "JPEG")
        # resize to 1277 by 1277
        image = Image.open(out_path)
    else:
        image = Image.open(img_path)

    new_image = image.resize((out_height, out_width))
    new_image.save(out_path)

    print(f"original size {image.size}")
    print(f"new size {new_image.size}") 
    image.close()

### 2. Feed Image into our Trained Convolutional Neural Network, PlastNet

In [ ]:
class PlastNet(nn.Module):
    name = "PlastNet"
    def __init__(self):
        # input dimensions: 1277x1277
        super(PlastNet, self).__init__()
        self.conv_layer_1 = nn.Sequential(
            nn.Conv2d(3, 16, 6), # 1272
            nn.BatchNorm2d(16), # must be output channel of previous layer
            nn.ReLU(inplace=True),
            nn.MaxPool2d(8, 8) # 159
        )
        # now size 636x636*16
        self.conv_layer_2 = nn.Sequential(
            nn.Conv2d(16, 16, 3), # 157
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2), # 78
            nn.Dropout2d(p=0.05),
        )
        # now size 317*317*16
        self.conv_layer_3 = nn.Sequential(
            nn.Conv2d(16, 32, 3), # 76
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2), # 38
        )
        # now size 157*157*32
        self.conv_layer_4 = nn.Sequential(
            nn.Conv2d(32, 32, 3), # 36
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2), # 18
            nn.Dropout2d(p=0.05), 
        )
        # now size 77*77*64
        self.conv_layer_5 = nn.Sequential(
            nn.Conv2d(32, 32, 3), # 16
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2), # 8
        )
        self.fc = nn.Sequential(
            nn.Linear(8*8*32, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        x = self.conv_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.conv_layer_3(x)
        x = self.conv_layer_4(x)
        x = self.conv_layer_5(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
model_path = "/content/gdrive/MyDrive/Praxis 3/Plastics-Tracking/CNN-Models/model_CNN_V0_bs32_lr0.001_epoch9"

model = PlastNet()
model.load_state_dict(torch.load(model_path))
model.eval()

PlastNet(
  (conv_layer_1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(6, 6), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer_2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.05, inplace=False)
  )
  (conv_layer_3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer_4): Sequential(
    (0): Conv2d(32, 32, kernel_size=(

In [ ]:
def classify_plastic(image_path):
    '''
    Take in a 1277x1277 rgb image and classify it as either PET or non-PET
    Return: a 2-element torch array with probabiliy of PET and non-PET
    '''
    im = plt.imread(image_path)
    im = np.transpose(im, [2,0,1])
    im = torch.from_numpy(im)
    im = im.type(torch.FloatTensor)
    # normalize = transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    # im = normalize(im)
    im = torch.unsqueeze(im, 0)

    pred = model(im)
    softmax = nn.Softmax(dim=1)
    pred = softmax(pred)
    print(f"PET: {pred[0][0]}, not PET: {pred[0][1]}")
    return pred

In [ ]:
source_path = "/content/gdrive/MyDrive/Praxis 3/Plastics-Tracking/Plastics-Images-Own/IMG_7342.HEIC"
out_path = "/content/gdrive/MyDrive/Praxis 3/Plastics-Tracking/Plastics-Images-Own-Resized/NP_IMG_7342.jpg"
preprocess(source_path, out_path, 200, 200)

original size (3024, 3024)
new size (200, 200)


In [ ]:
with torch.no_grad():
    wadaba_not_pet = "/content/gdrive/MyDrive/Praxis 3/Plastics-Tracking/WaDaBa-Processed/02-NOT-PET/0007_a06b01c2d0e0f0g0h2.jpg"
    
    # own_pet = "/content/gdrive/MyDrive/Praxis 3/Plastics-Tracking/Plastics-Images-Own-Resized/IMG_7332.jpg"
    # own_not_pet = "/content/gdrive/MyDrive/Praxis 3/Plastics-Tracking/Plastics-Images-Own-Resized/NP_IMG_7337.jpg"
    im = Image.open(out_path)
    plt.imshow(im)
    plt.axis('off')
    plt.title(out_path.split("/")[-1])
    classify_plastic(out_path)


PET: 0.0, not PET: 1.0


### Non-PET Things **correctly** classified as non-PET
- a bowl of soup (7344)
- an orange t-shirt (7346)


### Non-PET Things **incorrectly** classified
- Doritos bag
- shampoo bottle (HDPE instead of PET)
- calculator
- pencil case
- ipad
- battery
- transparent tape case
- stapler
- tide detergent bottle